In [1]:
import torch

if torch.cuda.is_available():
    print("CUDA is available. GPU will be used.")
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}") # Prints the name of the first GPU
else:
    print("CUDA is not available. CPU will be used.")

CUDA is available. GPU will be used.
Number of GPUs available: 1
GPU Name: Tesla P100-PCIE-16GB


In [ ]:
ls

In [ ]:
!unzip -qq stage1_train.zip -d stage1_train

In [ ]:
!unzip -qq untreated.zip -d untreated

In [ ]:
!unzip -qq paclitaxel.zip -d paclitaxel

In [ ]:
!unzip -qq vorinostat.zip -d vorinostat

In [ ]:
ls

In [2]:
import os
import numpy as np
from PIL import Image
import cv2
import shutil

In [ ]:
import os
import numpy as np
from PIL import Image
import cv2
import shutil

# Paths
DATASET_DIR = "stage1_train"
YOLO_DATASET_DIR = "bbbc050_yolo"

# YOLO format: one txt per image with lines: class_id x_center y_center width height (normalized)
os.makedirs(YOLO_DATASET_DIR + "/images/train", exist_ok=True)
os.makedirs(YOLO_DATASET_DIR + "/labels/train", exist_ok=True)

for img_id in os.listdir(DATASET_DIR):
    img_path = os.path.join(DATASET_DIR, img_id, "images", img_id + ".png")
    mask_dir = os.path.join(DATASET_DIR, img_id, "masks")

    # Copy image
    img_out = os.path.join(YOLO_DATASET_DIR, "images/train", img_id + ".png")
    shutil.copy(img_path, img_out)

    # Build YOLO labels
    H, W = 256, 256
    label_lines = []
    for mask_file in os.listdir(mask_dir):
        mask = np.array(Image.open(os.path.join(mask_dir, mask_file)).resize((W, H)))
        contours, _ = cv2.findContours((mask>127).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            # Normalize
            xc = (x + w/2) / W
            yc = (y + h/2) / H
            wn = w / W
            hn = h / H
            label_lines.append(f"0 {xc} {yc} {wn} {hn}\n")

    with open(os.path.join(YOLO_DATASET_DIR, "labels/train", img_id + ".txt"), "w") as f:
        f.writelines(label_lines)


In [4]:
from ultralytics import YOLO

# Load YOLOv8 nano (fastest) or small
model = YOLO("yolov8n.pt")

# Train
model.train(data="data.yaml", epochs=50, imgsz=256, batch=8)


New https://pypi.org/project/ultralytics/8.3.182 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.181 🚀 Python-3.10.18 torch-2.5.1 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train11, nbs=64, nms=False, opset=None, optimize=False, optimizer=au

train: Scanning /data/user/home/hnguye24/bbbc050_yolo/labels/train.cache... 670 images, 0 backgrounds, 0 corrupt: 100%|██████████| 670/670 [00:00<?, ?it/s]
/home/hnguye24/.conda/envs/yolo6/lib/python3.10/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4.3±1.8 MB/s, size: 116.7 KB)


val: Scanning /data/user/home/hnguye24/bbbc050_yolo/labels/train.cache... 670 images, 0 backgrounds, 0 corrupt: 100%|██████████| 670/670 [00:00<?, ?it/s]
/home/hnguye24/.conda/envs/yolo6/lib/python3.10/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Plotting labels to runs/detect/train11/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 256 train, 256 val
Using 8 dataloader workers
Logging results to runs/detect/train11
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        670      29475      0.907      0.771      0.836      0.579



50 epochs completed in 0.130 hours.
Optimizer stripped from runs/detect/train11/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train11/weights/best.pt, 6.2MB

Validating runs/detect/train11/weights/best.pt...
Ultralytics 8.3.181 🚀 Python-3.10.18 torch-2.5.1 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:11<00:00,  3.56it/s]


                   all        670      29475      0.909      0.773      0.835      0.581
Speed: 0.0ms preprocess, 0.6ms inference, 0.0ms loss, 7.8ms postprocess per image
Results saved to runs/detect/train11


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x2aacba166800>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [10]:
import matplotlib.pyplot as plt

# Run inference
results = model.predict(
    source=r"untreated/untreated/blue/B2AI_3_untreated_E1_R8_z01_blue.jpg",
    conf=0.5,
    save=False
)



# Create the plot
plt.figure(figsize=(8,8))
plt.imshow(results[0].plot())   # draws bounding boxes on image
plt.axis("off")
plt.title("YOLOv8 Detection Results")

# Save to file
plt.savefig("yolo_detection_output.png", bbox_inches="tight", dpi=300)

plt.close()  # close the figure (prevents duplicate display in notebooks)
print("Saved as yolo_detection_output.png")




image 1/1 /data/user/home/hnguye24/untreated/untreated/blue/B2AI_3_untreated_E1_R8_z01_blue.jpg: 256x256 10 nucleuss, 8.2ms
Speed: 0.8ms preprocess, 8.2ms inference, 1.6ms postprocess per image at shape (1, 3, 256, 256)
Saved as yolo_detection_output.png
